# by *convention* notebooks should be easy to re-use.

In [1]:
    ...; o = __name__ == '__main__'; ...;

Are single target `ast.Expr` that will `ast.literal_eval` is a possible parameter.

In [2]:
    from ast import NodeTransformer, parse, Assign, literal_eval, dump, fix_missing_locations, Str, Tuple
    from dataclasses import dataclass, field
    @dataclass
    class FreeStatement(NodeTransformer):
        params: list = field(default_factory=list)
        globals: dict = field(default_factory=dict)
        def visit_FunctionDef(FreeStatement, node): return node
        
        visit_ClassDef = visit_FunctionDef
                
        def visit_Assign(FreeStatement, node):
            if len(node.targets):
                try:
                    if not getattr(node.targets[0], 'id', '_').startswith('_'):
                        FreeStatement.globals[node.targets[0].id] = literal_eval(node.value)
                        return 
                except: assert True, """The target can not will not literally evaluate."""
            return node
                
        def __call__(FreeStatement, nodes): return FreeStatement.globals, fix_missing_locations(FreeStatement.visit(nodes))

# `Parameterize` notebooks

`Parameterize` is callable version of a notebook.  It uses `rites` to load the `NotebookNode` and evaluates the `FreeStatement`s to discover the signature.

In [3]:
    @dataclass(repr=False)
    class Parameterize:
        __file__: str = field(default=None, metadata="""A valid filename for the notebook is needed to import it.""")
        __ast__: any = field(default=None, metadata="""The module ast.""")
        __variables__: any = field(default_factory=dict, metadata="""The FreeStatement expression in the ast.""")
        __notebook__: dict = field(default_factory=dict, metadata="""A serialized notebook.""")
            
        def __post_init__(Parameterize):
            from IPython.utils.capture import capture_output
            from pathlib import Path
            Parameterize.__name__ = Path(Parameterize.__file__).stem
            with open(Parameterize.__file__) as f: 
                Parameterize.__notebook__ = __import__('nbformat').read(f, 4)       
            with capture_output(stdout=False, stderr=False) as output:
                Parameterize.__variables__, Parameterize.__ast__ = \
                    FreeStatement()(AST().from_notebook_node(Parameterize.__notebook__))
            Parameterize.__output__ = output
            Parameterize.__signature__ = Parameterize.vars_to_sig(**Parameterize.__variables__)
            Parameterize.__doc__ = docify(Parameterize.__notebook__)

        def __call__(Parameterize, **dict):
            Parameterize = __import__('copy').copy(Parameterize)
            Parameterize.__dict__.update(Parameterize.__variables__)
            Parameterize.__dict__.update(dict)
            exec(AST(filename=Parameterize.__file__).compile(Parameterize.__ast__), *[Parameterize.__dict__]*2)
            return Parameterize
        
        def interact(Parameterize): 
            """Use the ipywidgets.interact to explore the parameterized notebook."""
            return __import__('ipywidgets').interact(Parameterize)
        
        @staticmethod
        def vars_to_sig(**vars):
            """Create a signature for a dictionary of names."""
            from inspect import Parameter, Signature
            return Signature([Parameter(str, Parameter.KEYWORD_ONLY, default = vars[str]) for str in vars])
    
    try: from .rites import AST, docify, Partial
    except: from rites import AST, docify, Partial

#### Examples that do work

In [4]:
    param = 'xyz'
    extraparam = 42

#### Examples that do *not* work

In [5]:
    """Parameters are not created when literal_eval fails."""
    noparam0 = Parameterize
    
    """Multiple target assignments are ignored."""
    noparam1, noparam2 = 'xyz', 42

## Developer

In [6]:
    __test__ = dict(
        imports="""
        >>> assert callable(f)
        """,
        default="""
        >>> default = f()
        >>> assert default.param == default.noparam1 == 'xyz' and default.noparam2 == 42
        >>> assert all(str not in default.__signature__.parameters for str in ('noparam', 'noparam1', 'noparam2'))
        """,
        reuse="""
        >>> new = f(param=10)
        >>> assert new.param is 10 and new.extraparam is 42""",
    )
    if o:
        f = Parameterize(globals().get('__file__', 'param.ipynb'))
        __import__('doctest').testmod(verbose=1)

Trying:
    default = f()
Expecting nothing
ok
Trying:
    assert default.param == default.noparam1 == 'xyz' and default.noparam2 == 42
Expecting nothing
ok
Trying:
    assert all(str not in default.__signature__.parameters for str in ('noparam', 'noparam1', 'noparam2'))
Expecting nothing
ok
Trying:
    assert callable(f)
Expecting nothing
ok
Trying:
    new = f(param=10)
Expecting nothing
ok
Trying:
    assert new.param is 10 and new.extraparam is 42
Expecting nothing
ok
10 items had no tests:
    __main__
    __main__.FreeStatement
    __main__.FreeStatement.__call__
    __main__.FreeStatement.visit_Assign
    __main__.FreeStatement.visit_FunctionDef
    __main__.Parameterize
    __main__.Parameterize.__call__
    __main__.Parameterize.__post_init__
    __main__.Parameterize.interact
    __main__.Parameterize.vars_to_sig
3 items passed all tests:
   3 tests in __main__.__test__.default
   1 tests in __main__.__test__.imports
   2 tests in __main__.__test__.reuse
6 tests in 13 items.
